# P3: Building a Classifier - Titanic Survival Prediction

**Author:** Saratchandra Golla  
**Date:** 11/08/2025    
**Dataset:** Titanic (from seaborn)     
**Introduction:**
This project aims to build and evaluate machine learning classification models to predict passenger survival on the Titanic, using the publicly available seaborn Titanic dataset. We will compare the performance of three different classifier types—Decision Tree (DT), Support Vector Machine (SVM), and a Neural Network (NN)—across three distinct feature sets to determine the most effective approach for this prediction task. The entire process will follow a structured methodology: data preparation, feature selection, model training, and performance evaluation using metrics like accuracy, precision, recall, and F1-score.